In [13]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from time import time
from sklearn.metrics import precision_score,adjusted_rand_score,recall_score,f1_score,accuracy_score, normalized_mutual_info_score
import matplotlib.pyplot as plt

from istar.utils import save_image, load_image, save_tsv, write_string, load_pickle
from istar.rescale import rescale_image
from istar.preprocess import adjust_margins
from istar.extract_features import main_my as extract_features
from istar.get_mask import main_my as get_mask
from istar.select_genes import main_my as select_genes
from istar.impute import main_my as impute
from istar.cluster import main_my as cluster

import spatialid

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    dgl.random.seed(seed)

def detect_spatialid(train: ad.AnnData, test: ad.AnnData, random_state: int):
    seed_everything(random_state)
    
    spatial = spatialid.transfer.Transfer(spatial_data=test, single_data=train)
    spatial.learn_sc()
    y_score = spatial.annotation()
    
    ratio = 100.0 * (test.obs['anomaly_label'].value_counts()[1] / len(test.obs['anomaly_label']))
    thres = np.percentile(y_score, ratio)
    result = (y_score < thres).astype(int)
    return result

In [49]:
ref_dir = '/volume3/ylu/STANDS/hRCC_TLS/RCC_c2.h5ad'
adata_dir1 = '/volume3/ylu/STANDS/hRCC_TLS/RCC_c3.h5ad'
adata_dir2 = '/volume3/ylu/STANDS/hRCC_TLS/RCC_c4.h5ad'
n_cluster = 5
locs_id = 'spatial'

ref = sc.read_h5ad(ref_dir)

adata1 = sc.read_h5ad(adata_dir1)
adata1.obs['batch'] = 'batch1'
adata2 = sc.read_h5ad(adata_dir2)
adata2.obs['batch'] = 'batch2'

# concate the histology image
img1 = adata1.uns['spatial']['stomic']['images']['hires']
img2 = adata2.uns['spatial']['stomic']['images']['hires']

height = max(img1.shape[0], img2.shape[0])
width = max(img1.shape[1], img2.shape[1])
img1 = cv2.cvtColor(img1, cv2.COLOR_BGRA2BGR)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGRA2BGR)

canvas1 = np.zeros((height, width, 3)).astype(float)
canvas2 = np.zeros((height, width, 3)).astype(float)

canvas1[:img1.shape[0], :img1.shape[1], :] = img1
canvas2[:img2.shape[0], :img2.shape[1], :] = img2
img = np.hstack((canvas1, canvas2))
img = (img - img.min()) / (img.max() - img.min())
img = (img * 255).astype(np.uint8)

# adjust location
factor1 = adata1.uns['spatial']['stomic']['scalefactors']['tissue_hires_scalef']
factor2 = adata2.uns['spatial']['stomic']['scalefactors']['tissue_hires_scalef']
adata1.obsm[locs_id] = adata1.obsm[locs_id] * factor1
adata2.obsm[locs_id] = adata2.obsm[locs_id] * factor1
adata2.obsm[locs_id][:,0] += width

adata = ad.concat([adata1, adata2], merge='same')
adata

AnnData object with n_obs × n_vars = 7672 × 3000
    obs: 'cell_type', 'TLS', 'cell.type', 'anomaly_label', 'subtype_label', 'batch'
    obsm: 'X_pca', 'X_umap', 'spatial'
    layers: 'raw_count'

In [63]:
adata.obs['SpatialID'] = detect_spatialid(train=ref, test=adata, random_state=0)

In [52]:
d = './istar_data/'
sid = adata[adata.obs['SpatialID']==1]
# run combat
sc.pp.combat(sid, inplace = True)

Found 588 genes with zero variance.


View of AnnData object with n_obs × n_vars = 456 × 3000
    obs: 'cell_type', 'TLS', 'cell.type', 'anomaly_label', 'subtype_label', 'batch', 'spatialid'
    obsm: 'X_pca', 'X_umap', 'spatial'
    layers: 'raw_count'

In [53]:
locs = pd.DataFrame(sid.obsm[locs_id], index=sid.obs_names, columns=['x', 'y'])
cnts = pd.DataFrame(sid.X.toarray(), index=sid.obs_names, columns=sid.var_names)
# rescale image
pixel_size_raw = 0.25  # float(read_string(args.prefix+'pixel-size-raw.txt'))
pixel_size = 0.5  # float(read_string(args.prefix+'pixel-size.txt'))
scale = pixel_size_raw / pixel_size
# img = load_image(get_image_filename(args.prefix+'he-raw'))
img = np.array(img)
if img.ndim == 3 and img.shape[-1] == 4:
    img = img[..., :3]  # remove alpha channel
img = img.astype(np.float32)
# print(f'Rescaling image (scale: {scale:.3f})...')
t0 = time()
img = rescale_image(img, scale)
# print(int(time() - t0), 'sec')
img = img.astype(np.uint8)
save_image(img, d+'he-scaled.jpg')

# preprocess image
pad = 256
# load histology image
# img = load_image(d+'he-scaled.jpg')
# pad image with white to make dimension divisible by 256
img = adjust_margins(img, pad=pad, pad_value=255)
# save histology image
save_image(img, f'{d}he.jpg')

random_state = 0
extract_features(d, random_state=random_state)

get_mask(inpfile=d+'embeddings-hist.pickle', outfile=d+'mask-small.png')

r = 3
mask = load_image(d+'mask-small.png')
new_mask = np.zeros_like(mask, dtype=bool)
# locs = pd.DataFrame(sid.obsm['spatial'], index=sid.obs_names, columns=['x', 'y'])
locs = locs * scale
locs = locs.round().astype(int)
save_tsv(locs, d+'locs.tsv')
position = (locs / 16).round().astype(int)
for index, row in position.iterrows():
    x, y = row['x'], row['y']
    new_mask[y, x] = True
    for i in range(y - r, y + r + 1):
        for j in range(x - r, x + r + 1):
            if 0 <= i and 0 <= j:
                new_mask[i, j] = True
save_image(new_mask, d+'mask-small.png')

# select genes
n_top_genes = 3000
select_genes(d, cnts, n_top=n_top_genes)

radius = float(200)
radius = radius * scale
radius = np.round(radius).astype(int)
write_string(radius, d+'radius.txt')

# train gene expression prediction model and predict at super-resolution
impute(d, cnts=cnts, device='cuda')

# segment image by gene features
cluster(d, n_clusters = n_cluster)

labels = load_pickle(d+'clusters-gene/labels.pickle')

label = position.apply(lambda row: labels[row['y'], row['x']], axis=1)

./istar_data/he-scaled.jpg
./istar_data/he.jpg
Image loaded from ./istar_data/he.jpg
Smoothening cls embeddings...
runtime: 0
Smoothening sub embeddings...
runtime: 0
Saving embeddings...
0 sec
Embeddings saved to ./istar_data/embeddings-hist.pickle
Pickle loaded from ./istar_data/embeddings-hist.pickle
Clustering pixels using km...
(3072, 579)
1 sec
n_clusters: 2
./istar_data/mask-small.png
Image loaded from ./istar_data/mask-small.png
./istar_data/locs.tsv
./istar_data/mask-small.png
./istar_data/gene-names.txt
./istar_data/radius.txt
Pickle loaded from ./istar_data/embeddings-hist.pickle
Dataframe loaded from ./istar_data/locs.tsv
Image loaded from ./istar_data/he.jpg
x: (64, 128, 579) , y: (456, 3000)
minmax: (-3.9474597e-16, 1.9412891864786612)
./istar_data/states/00/training-data-plots/x0000.png
minmax: (0.0, 0.9968997240076528)
./istar_data/states/00/training-data-plots/y0000.png


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type        | Params
----------------------------------------
0 | net_lat | Sequential  | 345 K 
1 | net_out | FeedForward | 771 K 
----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.467     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

134 sec
Model saved to ./istar_data/states/00/model.pt
History saved to ./istar_data/states/00/history.pickle


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type        | Params
----------------------------------------
0 | net_lat | Sequential  | 345 K 
1 | net_out | FeedForward | 771 K 
----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.467     Total estimated model params size (MB)


...

131 sec
Model saved to ./istar_data/states/03/model.pt
History saved to ./istar_data/states/03/history.pickle


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type        | Params
----------------------------------------
0 | net_lat | Sequential  | 345 K 
1 | net_out | FeedForward | 771 K 
----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.467     Total estimated model params size (MB)


./istar_data/states/03/history.png
x: (64, 128, 579) , y: (456, 3000)
minmax: (-3.9474597e-16, 1.9412891864786612)
./istar_data/states/04/training-data-plots/x0000.png
minmax: (0.0, 0.9968997240076528)
./istar_data/states/04/training-data-plots/y0000.png


Training: 0it [00:00, ?it/s]

130 sec
Model saved to ./istar_data/states/04/model.pt
History saved to ./istar_data/states/04/history.pickle
./istar_data/states/04/history.png
Pickle loaded from ./istar_data/embeddings-gene.pickle
Image loaded from ./istar_data/mask-small.png
Smoothing embeddings...
gaussian filter: winsize=17, sigma=2.0
0 sec
Clustering pixels using km...
(2675, 256)
0 sec
n_clusters: 5
./istar_data/clusters-gene/labels.png
./istar_data/clusters-gene/masks/000.png
./istar_data/clusters-gene/masks/001.png
./istar_data/clusters-gene/masks/002.png
./istar_data/clusters-gene/masks/003.png
./istar_data/clusters-gene/masks/004.png
Pickle loaded from ./istar_data/clusters-gene/labels.pickle
